In [31]:
# Cell 1: Install required packages
!pip install -q sentence-transformers plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.9 MB/s eta 0:00:00


In [37]:
# Cell 2: Upload your FAQ dataset
from google.colab import files
uploaded = files.upload()

Saving customer_support_tickets.csv.zip to customer_support_tickets.csv (2).zip


In [40]:
# Cell 3: Load dataset
import pandas as pd

faq_df = None
for filename in uploaded.keys():
    faq_df = pd.read_csv(filename)
    break

# Print the column names to inspect them
print("DataFrame columns:", faq_df.columns.tolist())

# *** IMPORTANT ***
# If 'question' and 'answer' are not in the list above,
# you need to identify the correct column names and rename them.
# For example, if your columns are 'Question Text' and 'Answer Text',
# uncomment and modify the following lines:
# faq_df = faq_df.rename(columns={'Question Text': 'question', 'Answer Text': 'answer'})
# print("DataFrame columns after renaming:", faq_df.columns.tolist())


# Now, drop rows with missing values in the 'question' or 'answer' columns
faq_df = faq_df.dropna(subset=['Ticket Subject', 'Resolution']).reset_index(drop=True)
faq_df.head()

DataFrame columns: ['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Product Purchased', 'Date of Purchase', 'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status', 'Resolution', 'Ticket Priority', 'Ticket Channel', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating']


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
1,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
2,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0
3,11,Joseph Moreno,mbrown@example.org,48,Male,Nintendo Switch,2021-01-19,Cancellation request,Data loss,I'm having an issue with the {product_purchase...,Closed,Measure tonight surface feel forward.,High,Phone,2023-06-01 17:46:49,2023-05-31 23:51:49,1.0
4,12,Brandon Arnold,davisjohn@example.net,51,Male,Microsoft Xbox Controller,2021-10-24,Product inquiry,Software bug,I'm having an issue with the {product_purchase...,Closed,Measure there house management pick knowledge ...,High,Chat,2023-06-01 12:05:51,2023-06-01 09:27:51,1.0


In [42]:
# Cell 4: Compute embeddings for FAQ questions (using 'Ticket Subject')
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
faq_questions = faq_df['Ticket Subject'].astype(str).tolist()
faq_embeddings = model.encode(faq_questions, show_progress_bar=True)

Batches:   0%|          | 0/87 [00:00<?, ?it/s]

In [43]:
# Cell 5: Advanced semantic search chatbot logic (Ticket Subject <-> Resolution)
import numpy as np

chat_history = []
query_log = []

def find_best_answer(user_q, threshold=0.65):
    user_emb = model.encode([user_q])[0]
    sims = np.inner(faq_embeddings, user_emb)
    best_idx = np.argmax(sims)
    best_score = sims[best_idx]
    if best_score > threshold:
        return (
            faq_df.iloc[best_idx]['Resolution'],
            best_score,
            faq_df.iloc[best_idx]['Ticket Subject']
        )
    else:
        return "Sorry, I don't know the answer. Please contact support.", best_score, None

def chat():
    print('--- Advanced Customer Support Chatbot ---')
    while True:
        user_q = input('You: ')
        if user_q.lower() in ['exit', 'quit']:
            break
        answer, sim_score, matched_q = find_best_answer(user_q)
        chat_history.append({'user': user_q, 'bot': answer, 'score': sim_score, 'matched_q': matched_q})
        query_log.append(user_q)
        print(f'Bot: {answer}')
        if matched_q:
            print(f"(Matched FAQ: '{matched_q}' | Similarity: {sim_score:.2f})")
    print('Chat ended.')

In [48]:
# Cell 6: Start chatting
# Type your questions. Type 'exit' to end the chat.
chat()

--- Advanced Customer Support Chatbot ---
You: battery life
Bot: Series special into hour feel stuff.
(Matched FAQ: 'Battery life' | Similarity: 1.00)
You: hardware issue
Bot: However financial development significant camera job every.
(Matched FAQ: 'Hardware issue' | Similarity: 1.00)
You: payment issue
Bot: Eye while if exist name receive.
(Matched FAQ: 'Payment issue' | Similarity: 1.00)
You: Product recommendation 
Bot: Soldier we such inside.
(Matched FAQ: 'Product recommendation' | Similarity: 1.00)
You: exit
Chat ended.


In [50]:
# Cell 7: Visualize common queries and matched FAQs
import plotly.graph_objs as go
from collections import Counter

# Top 5 user queries
user_query_counts = Counter(query_log).most_common(5)
user_queries, user_counts = zip(*user_query_counts) if user_query_counts else ([], [])

# Top 5 matched FAQ questions
matched_questions = [entry['matched_q'] for entry in chat_history if entry['matched_q']]
matched_counts = Counter(matched_questions).most_common(5)
matched_qs, matched_q_counts = zip(*matched_counts) if matched_counts else ([], [])

# Plot
fig = go.Figure([
    go.Bar(x=user_queries, y=user_counts, name='User Queries'),
    go.Bar(x=matched_qs, y=matched_q_counts, name='Matched FAQs')
])
fig.update_layout(title='Top 5 User Queries and Matched FAQs', barmode='group', xaxis_title="Questions", yaxis_title="Count")
fig.show()